In [ ]:
%run console.py


In [ ]:
from api.src.clients.fmp_client import FMPClient
from api.src.utils.seed_adapters.run_seed_adapters import seed_sp500_data
from config import Config, DevelopmentConfig, TestingConfig, ProductionConfig 
import psycopg2

client = FMPClient()

In [5]:
import psycopg2
TEST_DB_HOST='localhost'
TEST_DB_NAME='public_companies_test_db' 
TEST_DB_USER='kielay'
TEST_DB_PASSWORD='Graci3dog88*'

conn = psycopg2.connect(
    dbname=TEST_DB_NAME,
    user=TEST_DB_USER,
    password=TEST_DB_PASSWORD,
    host=TEST_DB_HOST
)

cur = conn.cursor()

In [2]:


def build_from_record(Class, record):
    if not record: return None
    attr = dict(zip(Class.columns, record))
    obj = Class()
    obj.__dict__ = attr
    return obj

def build_from_records(Class, records):
   return [build_from_record(Class, record) for record in records]

def find_all(Class, cursor):
    sql_str = f"SELECT * FROM {Class.__table__}"
    cursor.execute(sql_str)
    records = cursor.fetchall()
    return [build_from_record(Class, record) for record in records]

def find(Class, id, cursor):
    sql_str = f"SELECT * FROM {Class.__table__} WHERE id = %s"
    cursor.execute(sql_str, (id,))
    record = cursor.fetchone()
    return build_from_record(Class, record)

def save(obj, conn, cursor):
    s_str = ', '.join(len(values(obj)) * ['%s'])
    obj_str = f"""INSERT INTO {obj.__table__} ({keys(obj)}) VALUES ({s_str});"""
    cursor.execute(obj_str, list(values(obj)))
    conn.commit()
    cursor.execute(f'SELECT * FROM {obj.__table__} ORDER BY id DESC LIMIT 1')
    record = cursor.fetchone()
    return build_from_record(type(obj), record)

# # alternate save method
# def save(obj, conn, cursor):
#     s_str = ', '.join(len(values(obj)) * ['%s'])
#     obj_str = f"""INSERT INTO {obj.__table__} ({keys(obj)}) VALUES ({s_str}) RETURNING {obj.columns[0]};""" #In this modification, the `INSERT` statement now includes a `RETURNING` clause that returns the value of the primary key column immediately after insertion. The `find` function is then used to retrieve the full record using the returned primary key value.}
#     cursor.execute(obj_str, list(values(obj)))
#     primary_key_value = cursor.fetchone()[0]
#     conn.commit()
#     return find(obj.__class__, primary_key_value, cursor)

# def save(obj, conn, cursor):
#     # Exclude the 'id' attribute if it exists since it's auto-generated
#     attr_to_save = {k: v for k, v in obj.__dict__.items() if k != 'id'}
    
#     # Construct the INSERT statement without the 'id'
#     columns_str = ', '.join(attr_to_save.keys())
#     values_str = ', '.join(['%s'] * len(attr_to_save))
    
#     # Create the SQL statement with RETURNING clause for 'id'
#     sql_str = f"INSERT INTO {obj.__table__} ({columns_str}) VALUES ({values_str}) RETURNING id;"
    
#     # Execute the statement and commit the changes
#     cursor.execute(sql_str, list(attr_to_save.values()))
#     obj_id = cursor.fetchone()[0] # Fetch the generated id
#     conn.commit()


def values(obj):
    venue_attrs = obj.__dict__
    return [venue_attrs[attr] for attr in obj.columns if attr in venue_attrs.keys()]

def keys(obj):
    venue_attrs = obj.__dict__
    selected = [attr for attr in obj.columns if attr in venue_attrs.keys()]
    return ', '.join(selected)

def drop_records(cursor, conn, table_name):
    cursor.execute(f"DELETE FROM {table_name};")
    conn.commit()

def drop_tables(table_names, cursor, conn):
    for table_name in table_names:
        drop_records(cursor, conn, table_name)

# def drop_all_tables(conn, cursor):
#     table_names = [] 
#     drop_tables(table_names, cursor, conn)

def find_by_name(Class, name, cursor):
    query = f"""SELECT * FROM {Class.__table__} WHERE name = %s """
    cursor.execute(query, (name,))
    record =  cursor.fetchone()
    obj = build_from_record(Class, record)
    return obj

def find_or_create_by_name(Class, name, conn, cursor):
    obj = find_by_name(Class, name, cursor)
    if not obj:
        new_obj = Class()
        new_obj.name = name
        obj = save(new_obj, conn, cursor)
    return obj

def find_or_build_by_name(Class, name, cursor):
    obj = Class.find_by_name(name, cursor)
    if not obj:
        obj = Class()
        obj.name = name
    return obj




OperationalError: connection to server on socket "/tmp/.s.PGSQL.5432" failed: FATAL:  database "kielay" does not exist


In [4]:
class Company:
    __table__ = 'companies'
    columns = ['id', 'name', 'ticker_symbol', 'industry_id', 'sector_id', 'exchange_id', 'location_id', 'cik', 'isEtf', 'isActivelyTrading']

    def __init__(self, **kwargs):
        for key in kwargs.keys():
            if key not in self.columns:
                raise ValueError(f'{key} not in {self.columns}')
        for k, v in kwargs.items():
            setattr(self, k, v)
            
microsoft = Company(**{'name': 'Microsoft Corporation', 'ticker_symbol': 'MSFT', 'industry_id': 1, 'sector_id': 1, 'exchange_id': 1})